In [2]:
# !pip install transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 bitsandbytes==0.40.2 safetensors>=0.3.1 tokenizers>=0.13.3
# !pip install word2number

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tqdm import tqdm
import pandas as pd
from preprocess import *

In [5]:
with open("/content/drive/My Drive/Datasets/Harvard n2c2 NLP Research Data Sets/2018 (Track 2) ADE and Medication Extraction Challenge/training_20180910/100035.txt", 'r') as f:
  text = f.read()
  print(text[10179:10197])

recurrent seizures


In [7]:
t_lines, t_stats, r_lines, r_stats = readAnnFile("/content/drive/My Drive/Datasets/Harvard n2c2 NLP Research Data Sets/2018 (Track 2) ADE and Medication Extraction Challenge/training_20180910/100035.ann")

In [8]:
len(t_lines)

292

In [9]:
t_lines2 = []
for line in tqdm(t_lines):
  if len(line) > 5:
    line.pop(3)
  t_lines2.append(line)

100%|██████████| 292/292 [00:00<00:00, 1296017.74it/s]


In [10]:
t_lines3 = {}
for line2 in t_lines2:
  t_lines3[line2[0]] = line2[1:]

In [11]:
r_lines2 = []
for r_l in r_lines:
  t1 = r_l[2].split(":")[1]
  t2 = r_l[3].split(":")[1]
  new_l = []
  new_l.extend(r_l[:2])
  new_l.extend(t_lines3[t1])
  new_l.extend(t_lines3[t2])
  r_lines2.append(new_l)

In [12]:
r_lines_df = pd.DataFrame(r_lines, columns=['red_id', 'red_type', 'rel_arg_1', 'rel_arg_2'])
r_lines_df.head()

,red_id,red_type,rel_arg_1,rel_arg_2
0,R1,Reason-Drug,Arg1:T1,Arg2:T3
1,R4,Route-Drug,Arg1:T5,Arg2:T3
2,R5,Strength-Drug,Arg1:T7,Arg2:T6
3,R6,Route-Drug,Arg1:T8,Arg2:T6
4,R7,Frequency-Drug,Arg1:T9,Arg2:T6


In [13]:
t_lines_df = pd.DataFrame(t_lines2, columns=['entity_id', 'entity_type', 'entity_start', 'entity_end', 'entity'])
t_lines_df.head()

,entity_id,entity_type,entity_start,entity_end,entity
0,T296,Drug,164,171,Vicodin
1,T344,Drug,739,742,CTX
2,T345,Drug,744,756,azithromycin
3,T346,Route,758,760,SC
4,T347,Drug,761,772,epinephrine


In [14]:
r_lines_df2 = pd.DataFrame(r_lines2, columns=['red_id', 'red_type', 'entity_1_type', 'entity_1_start', 'entity_1_end', 'entity_1', 'entity_2_type', 'entity_2_start', 'entity_2_end', 'entity_2'])
r_lines_df2.head()

,red_id,red_type,entity_1_type,entity_1_start,entity_1_end,entity_1,entity_2_type,entity_2_start,entity_2_end,entity_2
0,R1,Reason-Drug,Reason,10179,10197,recurrent seizures,Drug,10227,10233,ativan
1,R4,Route-Drug,Route,10240,10242,IM,Drug,10227,10233,ativan
2,R5,Strength-Drug,Strength,10466,10470,25mg,Drug,10455,10465,Topiramate
3,R6,Route-Drug,Route,10471,10473,PO,Drug,10455,10465,Topiramate
4,R7,Frequency-Drug,Frequency,10474,10477,BID,Drug,10455,10465,Topiramate


## Work on Creating Training Sentences

In [ ]:
import os
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_from_disk
import torch

import bitsandbytes as bnb
from huggingface_hub import login, HfFolder

model_id = "meta-llama/Llama-2-13b-hf" # sharded weights
model_id = model_id
dataset_path = '/opt/ml/input/data/training'
epochs = 3
per_device_train_batch_size = 2
lr = 2e-4
hf_token = HfFolder.get_token()
merge_weights = True


def parse_arge():
    """Parse the arguments."""
    parser = argparse.ArgumentParser()
    # add model id and dataset path argument
    parser.add_argument(
        "--model_id",
        type=str,
        help="Model id to use for training.",
    )
    parser.add_argument(
        "--dataset_path", type=str, default="lm_dataset", help="Path to dataset."
    )
    parser.add_argument(
        "--hf_token", type=str, default=HfFolder.get_token(), help="Path to dataset."
    )
    # add training hyperparameters for epochs, batch size, learning rate, and seed
    parser.add_argument(
        "--epochs", type=int, default=3, help="Number of epochs to train for."
    )
    parser.add_argument(
        "--per_device_train_batch_size",
        type=int,
        default=1,
        help="Batch size to use for training.",
    )
    parser.add_argument(
        "--lr", type=float, default=5e-5, help="Learning rate to use for training."
    )
    parser.add_argument(
        "--seed", type=int, default=42, help="Seed to use for training."
    )
    parser.add_argument(
        "--gradient_checkpointing",
        type=bool,
        default=True,
        help="Path to deepspeed config file.",
    )
    parser.add_argument(
        "--bf16",
        type=bool,
        default=True if torch.cuda.get_device_capability()[0] == 8 else False,
        help="Whether to use bf16.",
    )
    parser.add_argument(
        "--merge_weights",
        type=bool,
        default=True,
        help="Whether to merge LoRA weights with base model.",
    )
    args, _ = parser.parse_known_args()

    if args.hf_token:
        print(f"Logging into the Hugging Face Hub with token {args.hf_token[:10]}...")
        login(token=args.hf_token)

    return args


# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


# COPIED FROM https://github.com/artidoro/qlora/blob/main/qlora.py
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)


def create_peft_model(model, gradient_checkpointing=True, bf16=True):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )
    from peft.tuners.lora import LoraLayer

    # prepare int-4 model for training
    model = prepare_model_for_kbit_training(
        model, use_gradient_checkpointing=gradient_checkpointing
    )
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # get lora target modules
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        target_modules=modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )

    model = get_peft_model(model, peft_config)

    # pre-process the model by upcasting the layer norms in float 32 for
    for name, module in model.named_modules():
        if isinstance(module, LoraLayer):
            if bf16:
                module = module.to(torch.bfloat16)
        if "norm" in name:
            module = module.to(torch.float32)
        if "lm_head" in name or "embed_tokens" in name:
            if hasattr(module, "weight"):
                if bf16 and module.weight.dtype == torch.float32:
                    module = module.to(torch.bfloat16)

    model.print_trainable_parameters()
    return model


def training_function(args):
    # set seed
    set_seed(args.seed)

    dataset = load_from_disk(args.dataset_path)
    # load model from the hub with a bnb config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        args.model_id,
        use_cache=False
        if args.gradient_checkpointing
        else True,  # this is needed for gradient checkpointing
        device_map="auto",
        quantization_config=bnb_config,
    )

    # create peft config
    model = create_peft_model(
        model, gradient_checkpointing=args.gradient_checkpointing, bf16=args.bf16
    )

    # Define training args
    output_dir = "/tmp/llama2"
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=args.per_device_train_batch_size,
        bf16=args.bf16,  # Use BF16 if available
        learning_rate=args.lr,
        num_train_epochs=args.epochs,
        gradient_checkpointing=args.gradient_checkpointing,
        # logging strategies
        logging_dir=f"{output_dir}/logs",
        logging_strategy="steps",
        logging_steps=10,
        save_strategy="no",
    )

    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=default_data_collator,
    )

    # Start training
    trainer.train()

    sagemaker_save_dir="/opt/ml/model/"
    if args.merge_weights:
        # merge adapter weights with base model and save
        # save int 4 model
        trainer.model.save_pretrained(output_dir, safe_serialization=False)
        # clear memory
        del model
        del trainer
        torch.cuda.empty_cache()

        from peft import AutoPeftModelForCausalLM

        # load PEFT model in fp16
        model = AutoPeftModelForCausalLM.from_pretrained(
            output_dir,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
        )
        # Merge LoRA and base model and save
        model = model.merge_and_unload()
        model.save_pretrained(
            sagemaker_save_dir, safe_serialization=True, max_shard_size="2GB"
        )
    else:
        trainer.model.save_pretrained(
            sagemaker_save_dir, safe_serialization=True
        )

    # save tokenizer for easy inference
    tokenizer = AutoTokenizer.from_pretrained(args.model_id)
    tokenizer.save_pretrained(sagemaker_save_dir)


def main():
    args = parse_arge()
    training_function(args)


if __name__ == "__main__":
    main()